In [39]:
import pandas as pd

In [40]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\user\anaconda3\envs\assignment\lib\site-packages\tensorflow\python\client\session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [41]:
df = pd.read_csv("data.csv")

In [42]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [43]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [44]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [45]:
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [46]:
len(df)

569

In [47]:
df.drop('Unnamed: 32', axis = 1, inplace = True)
df.drop('id', axis = 1, inplace = True)

In [48]:
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [49]:
X = df.drop('diagnosis', axis = 1)

In [50]:
y = df['diagnosis']

In [51]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
y = labelencoder_X_1.fit_transform(y)

In [52]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [53]:

def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['acc'])
    return model

In [54]:
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='breast cancer1')

In [55]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
tuner.search(X_train, y_train,
             epochs=150,
             validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 55s]
val_acc: 0.9629629651705424

Best val_acc So Far: 0.9649122754732767
Total elapsed time: 00h 05m 16s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
num_layers        |9                 |6                 
units_0           |224               |128               
units_1           |64                |128               
learning_rate     |0.01              |0.01              
units_2           |32                |64                
units_3           |128               |64                
units_4           |64                |448               
units_5           |64                |256               
units_6           |480               |416               
units_7           |192               |224               
units_8           |256               |352               
units_9           |448               |64                
units_10          |320               |224               
units_11          |32                |192  

Epoch 52/150
13/13 [==============================] - 0s 10ms/step - loss: 0.2457 - acc: 0.9160 - val_loss: 0.2040 - val_acc: 0.9064
Epoch 53/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2165 - acc: 0.9175 - val_loss: 0.1418 - val_acc: 0.9649
Epoch 54/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1969 - acc: 0.9224 - val_loss: 0.1369 - val_acc: 0.9532
Epoch 55/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1498 - acc: 0.9400 - val_loss: 0.1519 - val_acc: 0.9415
Epoch 56/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1779 - acc: 0.9200 - val_loss: 0.1381 - val_acc: 0.9532
Epoch 57/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1594 - acc: 0.9364 - val_loss: 0.1407 - val_acc: 0.9532
Epoch 58/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1667 - acc: 0.9322 - val_loss: 0.2223 - val_acc: 0.8772
Epoch 59/150
13/13 [==============================] - 0s 9ms/step - los

Epoch 114/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1542 - acc: 0.9149 - val_loss: 0.1637 - val_acc: 0.9474
Epoch 115/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2241 - acc: 0.9078 - val_loss: 0.1280 - val_acc: 0.9415
Epoch 116/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1587 - acc: 0.9332 - val_loss: 0.1867 - val_acc: 0.9240
Epoch 117/150
13/13 [==============================] - 0s 10ms/step - loss: 0.2073 - acc: 0.9282 - val_loss: 0.1515 - val_acc: 0.9415
Epoch 118/150
13/13 [==============================] - 0s 14ms/step - loss: 0.2709 - acc: 0.9220 - val_loss: 0.1144 - val_acc: 0.9591
Epoch 119/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1550 - acc: 0.9383 - val_loss: 0.1428 - val_acc: 0.9649
Epoch 120/150
13/13 [==============================] - 0s 10ms/step - loss: 0.1930 - acc: 0.9154 - val_loss: 0.1192 - val_acc: 0.9649
Epoch 121/150
13/13 [==============================] - 0s 10ms/

13/13 [==============================] - 0s 9ms/step - loss: 0.1737 - acc: 0.9161 - val_loss: 0.2335 - val_acc: 0.8830
Epoch 26/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2546 - acc: 0.8894 - val_loss: 0.1883 - val_acc: 0.9181
Epoch 27/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1969 - acc: 0.9059 - val_loss: 0.1901 - val_acc: 0.9298
Epoch 28/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2142 - acc: 0.9005 - val_loss: 0.4171 - val_acc: 0.8070
Epoch 29/150
13/13 [==============================] - 0s 13ms/step - loss: 0.3825 - acc: 0.8681 - val_loss: 0.2020 - val_acc: 0.9532
Epoch 30/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2159 - acc: 0.9249 - val_loss: 0.1735 - val_acc: 0.9415
Epoch 31/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2651 - acc: 0.8930 - val_loss: 0.1513 - val_acc: 0.9591
Epoch 32/150
13/13 [==============================] - 0s 9ms/step - loss: 0.2087 - acc: 

13/13 [==============================] - 0s 9ms/step - loss: 0.1222 - acc: 0.9418 - val_loss: 0.2782 - val_acc: 0.8713
Epoch 88/150
13/13 [==============================] - 0s 10ms/step - loss: 0.2029 - acc: 0.9170 - val_loss: 0.1978 - val_acc: 0.9298
Epoch 89/150
13/13 [==============================] - 0s 10ms/step - loss: 0.2342 - acc: 0.9240 - val_loss: 0.1502 - val_acc: 0.9532
Epoch 90/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1342 - acc: 0.9517 - val_loss: 0.1505 - val_acc: 0.9532
Epoch 91/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1606 - acc: 0.9326 - val_loss: 0.1343 - val_acc: 0.9532
Epoch 92/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1141 - acc: 0.9399 - val_loss: 0.1193 - val_acc: 0.9532
Epoch 93/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1416 - acc: 0.9278 - val_loss: 0.1174 - val_acc: 0.9591
Epoch 94/150
13/13 [==============================] - 0s 9ms/step - loss: 0.1433 - acc:

13/13 [==============================] - 0s 13ms/step - loss: 0.6598 - acc: 0.6319 - val_loss: 0.6588 - val_acc: 0.6316
Epoch 149/150
13/13 [==============================] - 0s 12ms/step - loss: 0.6620 - acc: 0.6302 - val_loss: 0.6581 - val_acc: 0.6316
Epoch 150/150
13/13 [==============================] - 0s 13ms/step - loss: 0.6636 - acc: 0.6236 - val_loss: 0.6584 - val_acc: 0.6316
